# 17-01 OpenClaw 项目结构

深入理解 OpenClaw 的架构设计。

## 1. 整体架构

In [ ]:
// OpenClaw 项目结构
/*
myopenclaw/
├── src/                    # 核心源代码
│   ├── cli/               # CLI 命令行界面
│   ├── commands/          # CLI 命令实现
│   ├── channels/          # 消息通道（Telegram、Discord等）
│   ├── routing/           # 消息路由
│   ├── media/             # 媒体处理
│   └── infra/             # 基础设施
├── extensions/            # 插件扩展
├── apps/                  # 移动端/桌面端应用
├── ui/                    # UI 组件
└── docs/                  # 文档
*/

## 2. 核心模块分析

In [ ]:
// src/routing/message-router.ts
// 消息路由的核心逻辑

class MessageRouter {
  private handlers: Map<string, MessageHandler> = new Map();
  
  registerHandler(pattern: string, handler: MessageHandler) {
    this.handlers.set(pattern, handler);
  }
  
  async route(message: Message): Promise<Response> {
    // 1. 解析消息意图
    const intent = await this.parseIntent(message);
    
    // 2. 匹配处理器
    const handler = this.findHandler(intent);
    
    // 3. 执行处理
    return handler.handle(message, intent);
  }
}

// 查看实际源码
// cat src/routing/message-router.ts

## 3. 插件系统

In [ ]:
// extensions/ 目录结构
/*
extensions/
├── msteams/              # Microsoft Teams 插件
├── matrix/               # Matrix 协议插件
├── zalo/                 # Zalo 插件
└── voice-call/           # 语音通话插件
*/

// 插件接口定义
interface OpenClawExtension {
  name: string;
  version: string;
  
  // 初始化
  initialize(context: ExtensionContext): Promise<void>;
  
  // 注册处理器
  registerHandlers(router: MessageRouter): void;
  
  // 清理
  dispose(): Promise<void>;
}

## 4. 依赖注入模式

In [ ]:
// src/deps.ts
// OpenClaw 使用依赖注入管理组件

interface Deps {
  db: Database;
  redis: RedisClient;
  llm: LLMProvider;
  router: MessageRouter;
}

export function createDefaultDeps(): Deps {
  const db = createDatabase();
  const redis = createRedisClient();
  const llm = createLLMProvider();
  const router = new MessageRouter({ db, redis, llm });
  
  return { db, redis, llm, router };
}

// 测试时可以注入 Mock
const testDeps = {
  db: mockDatabase(),
  redis: mockRedis(),
  llm: mockLLM(),
  router: new MessageRouter(mockDeps)
};

## 5. 阅读源码建议

按以下顺序阅读：
1. `src/cli/` - 理解命令入口
2. `src/routing/` - 理解消息路由
3. `src/channels/` - 理解通道实现
4. `src/media/` - 理解媒体处理
5. `extensions/` - 理解插件系统

## 练习

1. 阅读 src/routing/message-router.ts
2. 分析一个 channel 的实现
3. 尝试添加一个简单的日志中间件